# 1 Knapsack Problem

The knapsack problem is one of the most trivial NP-hard problems. We can encounter many of its variants in the literature, which generally have various solution requirements for the algorithm. The formulation always contains criterium for the overall price, which means it is an optimization problem.

## 1.1 Author of Report

* **Name:** Tomáš Patro
* **Username:** patrotom
* **Email:** patrotom@fit.cvut.cz

## 1.2 Base of Problem

We are given

* integer $n$ (number of items)
* integer $M$ (capacity of knapsack)
* finite set $V = \{v_1, v_2, \dots, v_n\}$ (weights of items)
* finite set $C = \{c_1, c_2, \dots, c_n\}$ (prices of items)

## 1.3 Construction 0/1 Problem Form

The most well-known is the construction form. If we talk about the “knapsack problem” without a further description, we usually mean this version:

Construct the set $X = \{x_1, x_2, \dots, x_n\}$ where each $x_i$ is either $0$ or $1$, so $$v_1x_1 + v_2x_2 + \dots + v_nx_n \leq M \text{(knapsack is not overloaded)}$$ and expression $$c_1x_1 + c_2x_2 + \dots + c_nx_n$$ gained maximal value for all given sets (value of the items in the knapsack is maximal).

<br>

(“B201-NI-KOP: Problém Batohu (Knapsack Problem)” 2020)

# 2 Solution approaches

In the following section, we will describe approaches (algorithms) to solve the given problem:

* Brute force
* Branch and bound (B&B)
* Greedy heuristic
* Redux heuristic
* Dynamic programming (decomposition by weight and price)
* FPTAS algorithm

Brute force, B&B, and dynamic programming algorithms find the optimal price. Greedy, redux, and FPTAS algorithms may not found the optimal price, and we track the relative error of the computation. We compare the found price with the optimal price from the given solution files.

## 2.1 Brute Force

In the brute force approach, we search (almost) the whole state space to find the optimal price and configuration. The truth is that we do not go through the entire state space – that's why almost. We add a simple condition that cuts the recursion branch if adding an item would result in the overloaded knapsack.

Let us look at the code snippet of the algorithm:

``` python
def brute_force(conf, i, weight, price):
    if i == instance.size:
        if price >= solution.price:
            solution.price = price
            solution.weight = weight
            solution.conf = conf
        return

    new_weight = instance.items[i].weight + weight

    conf[i] = 1
    if new_weight <= instance.capacity:
        new_price = instance.items[i].price + price
        brute_force(conf, i + 1, new_weight, new_price)

    conf[i] = 0
    brute_force(conf, i + 1, weight, price)
```

The algorithm starts by comparing the current depth of the recursion `i` with the size of the instance. We use to comparison to determine if we have already solved one of the branches of recursion. If the `price` is greater or equal to the current optimal price, we update the `solution` object with the new values – `price`, `weight`, and vector configuration `conf`.

Next, we compute a new weight by adding the current weight to the particular items' weight. To cover the whole state space, we have to call the recursion with the current bit set to either $0$ or $1$. We also have to recalculate the price and weight. We later send them the recursion if the bit is set to $1$ – it means that the item is added to the knapsack. Also, as mentioned above, we call the recursion with the added item only if the recalculated weight is less than or equal to the capacity of the knapsack – we don't overload the knapsack.

## 2.2 Branch and Bound (B&B)

The beginning of the B&B algorithm is the same as in the brute force algorithm. The only difference is in the second condition for the cutting of the recursion calls (state space). Let us take a look at the difference in the algorithm:

``` python
def branch_and_bound(conf, i, weight, price):
    # The first condition is same as in the brute force algorithm...
    
    new_weight = instance.items[i].weight + weight
    new_price = instance.items[i].price + price
    upper_bound = price + instance.prices_sum(i=i)

    conf[i] = 1
    if (new_weight <= instance.capacity) and (upper_bound >= solution.price):
        branch_and_bound(conf, i + 1, new_weight, new_price)

    conf[i] = 0
    branch_and_bound(conf, i + 1, weight, price)
```

We enhance the condition by checking if the current `price` added to the sum of prices of the unvisited items (on the index `i` and greater) is greater or equal to the current optimal price. We can cut the recursion branch if we do not meet this condition. This way, we cut the branches of the state space, which do not lead us to the optimal price.

## 2.3 Greedy Heuristic

We implement a simple heuristic using the relation between the price and weight. Let us take a look at the algorithm:

``` python
def greedy():
    instance.sort_items()
    capacity = instance.capacity
    price = 0

    for item in instance.items:
        if capacity >= item.weight:
            price += item.price
            capacity -= item.weight
            solution.conf[item.index] = 1
    
    solution.price = price
```

We start by sorting the items in ascending order by $price/weight$ ratio. Then we iterate the items in this order. We check if the current capacity is greater or equal to the current items' weight in each iteration. If we meet this condition, we add the items' price to the overall price and lowers the capacity by the items' weight. We also update the configuration because we know that the item will be present in the result.

## 2.4 Redux Heuristic

Since we can encounter a situation where, for example, the last item alone would give us the optimal solution, we enhance the greedy heuristic in the following fashion:

``` python
def redux():
    greedy = Greedy()
    greedy.solve()
    highest_price, index = find_highest_price()

    if greedy.solution.price > highest_price:
        solution.price = greedy.solution.price
        solution.conf = greedy.solution.conf
    else:
        solution.price = highest_price
        solution.conf[index] = 1

def find_highest_price():
    highest_price = 0
    index = -1
    for item in instance.items:
        if item.price > highest_price and item.weight <= instance.capacity:
            highest_price = item.price
            index = item.index
    
    return (highest_price, index)
```

First, we calculate the price using the greedy algorithm. Next, we find the highest price of an item that does not overload the knapsack. Then, we compare the price found by the greedy algorithm with the highest price – the solution depends on the price, which is higher.

## 2.5 Dynamic Programming

Dynamic programming is always connected with a decomposition. We implement the following two decompositions by the overall:

* weight
* price

### 2.5.1 Weight Decomposition

The subproblems are re-evaluated, and the problem has overlapping subproblems property. We can take advantage of the dynamic programming by constructing a temporary 2D array `table` in a bottom-up manner:

``` python
def dynamic_by_weight():
    capacity = instance.capacity
    size = instance.size
    table = [[0 for x in range(capacity + 1)] for x in range(size + 1)]

    for i in range(size + 1):
        for w in range(capacity + 1):
            weight = instance.items[i - 1].weight
            price = instance.items[i - 1].price

            if i == 0 or w == 0:
                table[i][w] = 0
            elif weight <= w:
                table[i][w] = max(
                    price + table[i - 1][w - weight],
                    table[i - 1][w]
                )
            else:
                table[i][w] = table[i - 1][w]

    solution.price = table[size][capacity]
    solution.conf = construct_conf(capacity)

def construct_conf():
    conf = []

    for i in range(instance.size, 0, -1):
        if table[i][capacity] == table[i - 1][capacity]:
            conf.append(0)
        else:
            conf.append(1)
            capacity -= inst.items[i - 1].weight

    conf.reverse()

    return conf
```

In this solution, we take an approach where we compare the situation where the $n$th item is in the solution and when it is not. When we compare these two situations, we determine whether the item should or should not be in the solution. We repeat this process unless we encounter trivial subproblems. This approach leads us to an optimal solution. Afterward, we iterate over the table in a backward fashion to reconstruct the optimal solution's configuration.

### 2.5.2 Price Decomposition

Decomposition by the price is in a way similar to the decomposition by the weight. We change the `table`, so the sum of the prices composes the rows. Columns are composed of the items themselves. We can formalize the `table` in the following fashion:

$table(0,0) = 0$

$table(0, price) = \infty, \forall price > 0$

$table(i+1, price) = min(table(i, price), table(i, price - price_{i+1}) + weight_{i+1}), \forall i > 0$

``` python
def dynamic_by_price():
    prices_sum = instance.prices_sum()
    rows, cols = (prices_sum + 1, instance.size + 1)
    prepare_table(rows, cols)

    for i in range(1, cols):
        for j in range(1, rows):
            left = table[i - 1][j]
            left_b = table[i - 1][j - instance.items[i - 1].price]
            table[i][j] = min(
                left,
                left_b + instance.items[i - 1].weight
            )

    solution.price = find_optimal_price(prices_sum)
    solution.conf = construct_conf(solution.price)

def prepare_table():
    table = [
        [None if x > 0 else 0 for x in range(rows)] if x > 0 \
            else [infinity if x > 0 else 0 for x in range(rows)] \
                for x in range(cols)
    ]

def find_optimal_price(prices_sum):
    price = 0

    for i, w in enumerate(reversed(table[instance.size])):
        if w is not None and 0 < w <= instance.capacity:
            price = prices_sum - i
            break
    
    return price

def construct_conf(price):
    conf = []

    for i in range(instance.size, 0, -1):
        if table[i][price] == table[i - 1][price]:
            conf.append(0)
        else:
            conf.append(1)
            price -= instance.items[i - 1].price

    conf.reverse()

    return conf
```

We start by preparing the temporary 2D array `table`. Then, we iterate over this 2D array and fill it with the values which enable us to find the optimal price. Afterward, we find the optimal price by iterating over the prepared `table`. We reconstruct the configuration in a similar fashion as in the decomposition by the weight.

## 2.6 FPTAS Algorithm

Let us define:

* $P(S)$ optimal value criterium of the solution $S$
* $APR(I)$ approx. solution of the instance $I$
* $OPT(I)$ optimal solution of the instance $I$

$APR$ has a relative quality $R$, if

* $\forall I: R \geq max\{\frac{P(APR(I))}{P(OPT(I))}, \frac{P(OPT(I)}{P(APR(I))}\}$.

$APR$ has a relative error $\epsilon$, if

* $\forall I: \epsilon \geq max\{\frac{|P(APR(I)) - P(OPT(I))|}{max\{P(OPT(I)),P(APR(I))\}}\}$

$APR$, which $\forall \epsilon: 1 \gt \epsilon \gt 0$ solves each instance of the problem with the maximal relative error $\epsilon$ in polynomial time $|I|$ as instance size, we call **Polynomial Time Approximation Scheme (PTAS)**.

PTAS, which time depends polynomially on $\frac{1}{\epsilon}$ we call **Fully Polynomial Approximation Scheme (FTPAS)**.

Based on the formal description above, we implement the algorithm in the following fashion:

``` python
def fptas():
    max_price = max(instance.items, key=lambda x: x.price).price
    k = (instance.epsilon * max_price) / instance.size
    original_prices = list(map(lambda x: x.price, instance.items))
    instance.floor_prices(k)

    dynamic_price = DynamicPrice()
    dynamic_price.solve()

    solution.conf = dynamic_price.solution.conf
    solution.time = dynamic_price.solution.time
    solution.price = sum(
        [p for i, p in enumerate(original_prices) if solution.conf[i] == 1]
    )
```

We start by calculating the maximum price of the items. Then, we calculate the approximation variable $k$. We floor the prices of the instance using the variable $k$. We compute the optimal price using the floored prices and dynamic programming decomposed by price. Then, we calculate the resulting price by iterating over the solution configuration using the original prices.

# Results Analysis

We analyze the data using the *Python* language and its mathematical modules. The graphs and the whole report are rendered in the *Jupyter Notebook* – that's why we also add the code snippets in this report, which process data and generate the graphs. Data generated by the solution code are stored in the CSV format files, so we can easily load and process them.

We start by loading and processing the data:

# Bibliography

<br>

“B201-NI-KOP: Problém Batohu (Knapsack Problem).” 2020. Accessed October 31. [https://
moodle-vyuka.cvut.cz/mod/page/view.php?id=89694]().